In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *

In [ ]:
casenames = dict(
    c1 = 'R11_MSM97-CT_f0_fixdt3',
    # c2 = 'R11_MSM97-CT_Stokes6_f0_fixdt3',
    c3 = 'R11_MSM97-CT_Stokes_f0_fixdt3',
    # c4 = 'R11_MSM97-CT_Stokes5_f0_fixdt3',
)
amp = dict(
    c1 = 0.0,
    c2 = 0.45,
    c3 = 0.8,
    c4 = 1.423,
)
abc = 'abc'
titles = ['$a$ = {:4.2f} m'.format(amp[cn]) for cn in casenames.keys()]
iend = 57600
filename = 'his.mp.vis.000001.{:06d}.nc'.format(iend+1)
figpath = 'R11_MSM97_CT_Stokes_f0'
os.makedirs(figpath, exist_ok=True)

In [ ]:
ds = {}
for cn in casenames.keys():
    print(cn)
    datapath = os.path.join(os.path.pardir, 'ncarles', casenames[cn], filename)
    ds[cn] = NCARLESDataProfile(filepath=datapath).dataset 

In [ ]:
ds['c3']

In [ ]:
tslice = slice('2000-01-01T12:00:00', '2000-01-02T00:00:00')
# tslice = slice('2000-01-02T00:00:00', '2000-01-03T00:00:00')

In [ ]:
g = 9.81
alpha = 2e-4
Qt0 = 1.221e-5
B0 = alpha*g*Qt0
depth = 50

def get_tke(dss, a):
    tke_diss = -dss.data_vars['t_dsle']
    tke_pstk = dss.data_vars['t_stokes']
    tke_buoy = dss.data_vars['t_buoy']
    tke_tran = dss.data_vars['t_tran']
    tke_prod = dss.data_vars['t_rprod']
    tke_budget = dict(
        prod = tke_prod/B0,
        pstk = tke_pstk/B0,
        buoy = tke_buoy/B0,
        # tran = -(tke_diss+tke_buoy+tke_pstk+tke_prod)/B0,
        tran = tke_tran/B0,
        # resi = (tke_diss + tke_buoy + tke_tran + tke_pstk + tke_prod)/B0,
        # diss = tke_diss/B0,
        diss = -(tke_buoy + tke_tran + tke_pstk + tke_prod)/B0,
    )
    return tke_budget


colors = dict(
    prod = 'tab:blue',
    pstk = 'tab:red',
    diss = 'tab:purple',
    tran = 'tab:green',
    buoy = 'tab:orange',
    resi = 'k'
)

labels = dict(
    prod = '$P$',
    pstk = '$P^S$',
    diss = '$\\varepsilon$',
    tran = '$T$',
    buoy = '$B$',
    resi = 'res',
)

In [ ]:
if len(casenames) == 2:
    fig, axarr = plt.subplots(1, 2, sharey='row')
    fig.set_size_inches(5.5,2.8)
    for i, cname in enumerate(casenames.keys()):
        ax = axarr[i]
        tke_budget = get_tke(ds[cname], amp[cname])
        for term in tke_budget.keys():
            tke_budget[term].sel(time=tslice).mean(dim='time').plot(ax=ax, y='zi', color=colors[term])
        ax.set_xlim([-1,1])
        ax.set_ylim([-depth,0])
        ax.set_ylabel('')
        ax.set_xlabel('TKE Budget')
        ax.text(0.95, 0.05, '({:s})'.format(abc[i]), transform=ax.transAxes, va='bottom', ha='right')
        ax.grid()
        ax.set_title(titles[i], fontsize=10)
    axarr[0].set_ylabel('$z$ [m]')

    ax = axarr[-1]
    for term in tke_budget.keys():
        ax.plot(np.nan, np.nan, color=colors[term], label=labels[term])
    ax.legend(loc='lower left')

    plt.tight_layout()
    figname = os.path.join(figpath, 'profiles-ncar-cmp-tke-2c')
    fig.savefig(figname, dpi = 300, facecolor='w')

In [ ]:
if len(casenames) == 3:
    fig, axarr = plt.subplots(1, 3, sharey='row')
    fig.set_size_inches(8,3)
    for i, cname in enumerate(casenames.keys()):
        ax = axarr[i]
        tke_budget = get_tke(ds[cname], amp[cname])
        for term in tke_budget.keys():
            tke_budget[term].sel(time=tslice).mean(dim='time').plot(ax=ax, y='zi', color=colors[term])
        ax.set_xlim([-1,1])
        ax.set_ylim([-depth,0])
        ax.set_ylabel('')
        ax.set_xlabel('TKE Budget')
        ax.text(0.95, 0.05, '({:s})'.format(abc[i]), transform=ax.transAxes, va='bottom', ha='right')
        ax.grid()
        ax.set_title(titles[i], fontsize=10)
    axarr[0].set_ylabel('$z$ [m]')

    ax = axarr[-1]
    for term in tke_budget.keys():
        ax.plot(np.nan, np.nan, color=colors[term], label=labels[term])
    ax.legend(loc='lower left')

    plt.tight_layout()
    figname = os.path.join(figpath, 'profiles-ncar-cmp-tke')
    fig.savefig(figname, dpi = 300, facecolor='w')

In [ ]:
if len(casenames) == 4:
    fig, axarr = plt.subplots(2, 2, sharey='row', sharex='col')
    fig.set_size_inches(5.5,5.5)
    for i, cname in enumerate(casenames.keys()):
        m = i//2
        n = i%2
        ax = axarr[m,n]
        tke_budget = get_tke(ds[cname], amp[cname])
        for term in tke_budget.keys():
            tke_budget[term].sel(time=tslice).mean(dim='time').plot(ax=ax, y='z', color=colors[term])
        ax.set_xlim([-1,1])
        ax.set_ylim([-depth,0])
        ax.set_ylabel('')
        if m == 1:
            ax.set_xlabel('TKE Budget')
        else:
            ax.set_xlabel('')
        ax.text(0.95, 0.05, '({:s})'.format(abc[i]), transform=ax.transAxes, va='bottom', ha='right')
        ax.grid()
        ax.set_title(titles[i], fontsize=10)
    for i in np.arange(2):
        axarr[i,0].set_ylabel('$z$ [m]')

    ax = axarr[-1,-1]
    for term in tke_budget.keys():
        ax.plot(np.nan, np.nan, color=colors[term], label=labels[term])
    ax.legend(loc='lower left')

    plt.tight_layout()
    figname = os.path.join(figpath, 'profiles-ncar-cmp-tke-4c')
    fig.savefig(figname, dpi = 300, facecolor='w')